In [2]:
# Install Hugging Face libraries
!pip install transformers==4.30.2 datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
  Attempting uninstall:

In [ ]:
!pip show transformers


Name: transformers
Version: 4.30.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, sentence-transformers


In [3]:
import pandas as pd

# Replace 'your_file.csv' with the name of the file you uploaded
df = pd.read_csv('/content/text_to_sql_large_dataset_fineTuned.csv')

# Check the first few rows to verify the data
df.head()

,input_text,output_text
0,List all homes that have exactly 1 bathrooms.,SELECT * FROM listings WHERE bathrooms LIKE 1;
1,List all listings in San Francisco.,SELECT * FROM listings WHERE city = 'San Franc...
2,List all homes that have exactly 1 bathrooms.,SELECT * FROM listings WHERE bathrooms = 1;
3,Show listings priced more than $1000.,SELECT * FROM listings WHERE price > 1000;
4,Show listings priced less than $100.,SELECT * FROM listings WHERE price < 100;


In [4]:
import os
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Trainer, TrainingArguments

# Disable W&B logging
os.environ["WANDB_DISABLED"] = "true"

# Load the tokenizer and model
#tokenizer = AutoTokenizer.from_pretrained("suriya7/t5-base-text-to-sql")
#model = AutoModelForSeq2SeqLM.from_pretrained("suriya7/t5-base-text-to-sql")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/fine_tuned_model_ex")
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/fine_tuned_model_ex")

# Convert the DataFrame to HuggingFace Dataset format
dataset = Dataset.from_pandas(df)

# Tokenize the data
def tokenize_function(examples):
    model_inputs = tokenizer(examples['input_text'], max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(examples['output_text'], max_length=512, truncation=True, padding="max_length")
    labels = labels.input_ids
    labels = [-100 if token == tokenizer.pad_token_id else token for token in labels]  # Ignore pad tokens
    model_inputs["labels"] = labels
    return model_inputs

# Apply the tokenization to the entire dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Set up the data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Set up the training arguments without W&B
training_args = TrainingArguments(
    output_dir="./results",          # Where to store the model checkpoints
    evaluation_strategy="epoch",           # Use the new `eval_strategy` parameter
    learning_rate=3e-5,              # Learning rate
    per_device_train_batch_size=16,   # Batch size for training
    per_device_eval_batch_size=16,    # Batch size for evaluation
    num_train_epochs=3,               # Number of epochs
    weight_decay=0.01,               # Strength of weight decay
    save_steps=1000,                 # Save checkpoints every 1000 steps
    logging_dir="./logs",            # Directory for logging
    logging_steps=10,               # Log every 500 steps
    run_name="text-to-sql-training"  # Optional: You can still give a run name without W&B
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets,  # Use a separate validation dataset if available
    data_collator=data_collator
)

# Start training
trainer.train()

# Save the fine-tuned model
model.save_pretrained("fine_tuned_model")
tokenizer.save_pretrained("fine_tuned_model")


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:463: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location="cp

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.000500,0.000378
2,0.000400,0.000364
3,0.000500,0.000344


('fine_tuned_model/tokenizer_config.json',
 'fine_tuned_model/special_tokens_map.json',
 'fine_tuned_model/spiece.model',
 'fine_tuned_model/added_tokens.json',
 'fine_tuned_model/tokenizer.json')

In [10]:
import shutil

# Zip the model directory
shutil.make_archive("/content/fine_tuned_model", 'zip', "./fine_tuned_model")


'/content/fine_tuned_model.zip'

In [11]:
from google.colab import files
files.download("/content/fine_tuned_model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Path to your fine-tuned model directory
model_path = "/content/fine_tuned_model"  # Adjust this to the correct path in Colab

# Load the fine-tuned model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Check if model and tokenizer loaded successfully
print("Model and Tokenizer Loaded Successfully")


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:463: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location="cp

Model and Tokenizer Loaded Successfully


In [6]:
def generate_sql_query(natural_language_query):
    # Prepare the input text for translation (from English to SQL)
    input_text = f"translate English to SQL: {natural_language_query}"

    # Tokenize the input text
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    # Generate the SQL query
    output_ids = model.generate(input_ids)

    # Decode output to get the SQL query
    sql_query = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return sql_query

# Test the model with a sample query
query = "Show me all listings in New York City"
generated_sql = generate_sql_query(query)
print(f"Generated SQL Query: {generated_sql}")

query = "Find all listings with more than 3 bedrooms"
generated_sql = generate_sql_query(query)
print(f"Generated SQL Query: {generated_sql}")


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Generated SQL Query: SELECT * FROM listings WHERE city = 'New York City';
Generated SQL Query: SELECT * FROM listings WHERE bedrooms > 3;


In [7]:
query = "Find listings with more than 4 bathrooms and a priced above $330"
generated_sql = generate_sql_query(query)
print(f"Generated SQL Query: {generated_sql}")

Generated SQL Query: SELECT * FROM listings WHERE bathrooms > 4; price > 330;


In [8]:
query = "Show all listings in Los Angeles, CA, United States and have a price below $73"
generated_sql = generate_sql_query(query)
print(f"Generated SQL Query: {generated_sql}")

Generated SQL Query: SELECT * FROM listings WHERE city = 'Los Angeles, CA


In [9]:
query = "Find all listings in Agoura Hills with a price below $905"
generated_sql = generate_sql_query(query)
print(f"Generated SQL Query: {generated_sql}")

Generated SQL Query: SELECT * FROM listings WHERE city = 'Agoura Hills';


In [12]:
pip install rouge-score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=4d5d6b71e9a5c69921b901408b4750c390c417f1b7bf5f146f5a3c745720343d
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [13]:
import pandas as pd

# Replace 'your_file.csv' with the name of the file you uploaded
df_pred = pd.read_csv('/content/sample_text_to_sql_dataset.csv')

# Check the first few rows to verify the data
df_pred.head()

,input_text,output_text
0,Find all listings with more than 4 bedrooms,SELECT * FROM listings WHERE bedrooms > 4
1,Show all listings in Duarte priced below $491,SELECT * FROM listings WHERE price < 491 AND c...
2,Find properties with at least 2 bedrooms in Sa...,SELECT * FROM listings WHERE bedrooms >= 2 AND...
3,Find properties with at least 3 bedrooms in 马里布,SELECT * FROM listings WHERE bedrooms >= 3 AND...
4,Find all listings with more than 1 bedrooms,SELECT * FROM listings WHERE bedrooms > 1


In [14]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import Dataset
from rouge_score import rouge_scorer
import torch

# Load the fine-tuned model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("fine_tuned_model")
model = AutoModelForSeq2SeqLM.from_pretrained("fine_tuned_model")

# Convert the DataFrame to a HuggingFace Dataset format (use your dataset DataFrame here)
dataset = Dataset.from_pandas(df_pred)

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Function to generate model predictions and compute ROUGE scores
def evaluate_rouge(dataset, model, tokenizer):
    total_scores = {"rouge1": 0, "rouge2": 0, "rougeL": 0}
    num_samples = 0

    for sample in dataset:
        input_text = sample['input_text']
        expected_output = sample['output_text']

        # Tokenize input and generate predictions
        inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)
        with torch.no_grad():
            outputs = model.generate(**inputs, max_length=512)

        # Decode model prediction
        predicted_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Compute ROUGE scores
        rouge_score = scorer.score(expected_output, predicted_output)
        total_scores["rouge1"] += rouge_score['rouge1'].fmeasure
        total_scores["rouge2"] += rouge_score['rouge2'].fmeasure
        total_scores["rougeL"] += rouge_score['rougeL'].fmeasure
        num_samples += 1

        # Print individual sample scores (optional)
        print("Input Text:", input_text)
        print("Expected Output:", expected_output)
        print("Predicted Output:", predicted_output)
        print(f"ROUGE-1 F1: {rouge_score['rouge1'].fmeasure:.4f}, ROUGE-2 F1: {rouge_score['rouge2'].fmeasure:.4f}, ROUGE-L F1: {rouge_score['rougeL'].fmeasure:.4f}")
        print("-" * 80)

    # Calculate average scores
    avg_scores = {k: v / num_samples for k, v in total_scores.items()}
    return avg_scores

# Evaluate the dataset for ROUGE scores
average_rouge_scores = evaluate_rouge(dataset, model, tokenizer)

# Print average scores
print("\nAverage ROUGE Scores:")
print(f"ROUGE-1 F1: {average_rouge_scores['rouge1']:.4f}")
print(f"ROUGE-2 F1: {average_rouge_scores['rouge2']:.4f}")
print(f"ROUGE-L F1: {average_rouge_scores['rougeL']:.4f}")


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:463: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location="cp

Streaming output truncated to the last 5000 lines.
Input Text: Show all listings in Duarte priced below $491
Expected Output: SELECT * FROM listings WHERE price < 491 AND city = 'Duarte'
Predicted Output: SELECT * FROM listings WHERE city = 'Duarte'; price  491;
ROUGE-1 F1: 0.9412, ROUGE-2 F1: 0.6667, ROUGE-L F1: 0.7059
--------------------------------------------------------------------------------
Input Text: Find properties with at least 2 bedrooms in San Dimas
Expected Output: SELECT * FROM listings WHERE bedrooms >= 2 AND city = 'San Dimas'
Predicted Output: SELECT * FROM listings WHERE city = 'San Dimas';
ROUGE-1 F1: 0.8235, ROUGE-2 F1: 0.6667, ROUGE-L F1: 0.8235
--------------------------------------------------------------------------------
Input Text: Find properties with at least 3 bedrooms in 马里布
Expected Output: SELECT * FROM listings WHERE bedrooms >= 3 AND city = '马里布'
Predicted Output: SELECT * FROM listings WHERE city = '';
ROUGE-1 F1: 0.7692, ROUGE-2 F1: 0.5455, ROUGE-

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
